In [8]:
import pandas as pd
import numpy as np
combined_data = pd.read_csv('../data/processed/final_merged.csv', low_memory=False)

In [9]:
combined_data.count()

id            46985090
item_id       46985090
dept_id       46985090
cat_id        46985090
store_id      46985090
state_id      46985090
date          46985090
units_sold    46985090
wm_yr_wk      46985090
sell_price    46985090
sales         46985090
dtype: int64

In [10]:
combined_data.head(30491)

,id,item_id,dept_id,cat_id,store_id,state_id,date,units_sold,wm_yr_wk,sell_price,sales
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-29,3,11101,2.00,6.00
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-30,0,11101,2.00,0.00
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-31,0,11101,2.00,0.00
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-01,1,11101,2.00,2.00
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-02,4,11101,2.00,8.00
...,...,...,...,...,...,...,...,...,...,...,...
30486,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-20,3,11416,0.98,2.94
30487,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-21,5,11416,0.98,4.90
30488,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-22,3,11416,0.98,2.94
30489,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-23,5,11416,0.98,4.90


In [13]:
count_rows = combined_data[(combined_data['item_id'] == 'FOODS_1_021') & (combined_data['store_id'] == 'CA_1')].shape[0]
print(count_rows)

1541


In [15]:
unique_item_count = len(target_item_ids)
print("Number of unique item_ids:", unique_item_count)


Number of unique item_ids: 3049


In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load your dataset (assuming it's stored in a DataFrame named 'combined_data')
# Make sure your 'date' column is in datetime format
combined_data['date'] = pd.to_datetime(combined_data['date'])

# Select the features (independent variables) and target (sales) for modeling
# Here, we're using 'sell_price' and 'units_sold' as features, but you can choose other relevant features.
features = ['sell_price', 'units_sold']
target = 'sales'

X = combined_data[features]
y = combined_data[target]

# Initialize and train a Linear Regression model
model = LinearRegression()
model.fit(X, y)

# Make predictions on the entire dataset
y_pred = model.predict(X)

# Evaluate the model's performance
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)
r2 = r2_score(y, y_pred)

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")

# Optionally, you can save the trained model for future use
# from joblib import dump
# dump(model, 'sales_predictor_model.joblib')


Mean Absolute Error (MAE): 2.3595642171467914
Mean Squared Error (MSE): 36.552552989755604
Root Mean Squared Error (RMSE): 6.0458707387567925
R-squared (R2) Score: 0.5653557820777411


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Load your dataset (assuming it's in a CSV format)
data = pd.read_csv("your_dataset.csv")

# Subset your data to the first 500 rows
subset_data = data.iloc[:500]

# Extract the sales data for each item as a time series
sales_data = subset_data.iloc[:, 6:]  # Assuming the sales columns start from d_1 onwards

# Transpose the sales data to have time steps as rows and items as columns
sales_data = sales_data.T

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
sales_data_scaled = scaler.fit_transform(sales_data)

# Prepare the target variable (e.g., sales for a specific item)
target_item_id = "HOBBIES_1_001_CA_1_evaluation"  # Replace with the item you want to predict
y = sales_data_scaled[sales_data.index == target_item_id].flatten()

# Create input sequences for the LSTM model
sequence_length = 30  # You can adjust this based on your data
X = []
y = y[sequence_length:]  # Align the target variable with the sequences

for i in range(len(y) - sequence_length):
    X.append(sales_data_scaled[:, i:i+sequence_length])

X = np.array(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Plot actual vs. predicted sales
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('Actual vs. Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.show()

# Now you can use this trained LSTM model for sales predictions on new data


In [ ]:
# Load your dataset (assuming it's in a CSV format)
df = pd.read_csv('your_data.csv', low_memory=False)

# Extract relevant features and target
features = ['sell_price', 'units_sold']  # Add other relevant features
target = 'sales'

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Define the sequence length and create input sequences for LSTM
sequence_length = 30  # You can adjust this based on your data
X_seq, y_seq = [], []

for i in range(len(df) - sequence_length):
    X_seq.append(df[features][i:i+sequence_length].values)
    y_seq.append(df[target][i+sequence_length])

X_seq, y_seq = np.array(X_seq), np.array(y_seq)

# Create and train the LSTM model (as shown in the previous code)

# Function to predict sales for a specific item and store on a given date
def predict_sales(item_id, store_id, target_date):
    # Get historical data for the specified item and store
    item_data = df[(df['item_id'] == item_id) & (df['store_id'] == store_id)]
    
    # Filter data for the target date
    target_data = item_data[item_data['date'] == target_date]
    
    # Extract the input sequence for prediction
    input_sequence = target_data[features].tail(sequence_length).values
    
    # Make a prediction using the trained LSTM model
    predicted_sales = model.predict(np.array([input_sequence]))
    
    return predicted_sales

# Example usage:
item_id = 'your_item_id'
store_id = 'your_store_id'
target_date = 'your_target_date'  # Format: 'YYYY-MM-DD'

predicted_sales = predict_sales(item_id, store_id, target_date)
print(f"Predicted sales for {item_id} at {store_id} on {target_date}: {predicted_sales[0][0]}")
